<div style='background-image: url("header.png") ; padding: 0px ; background-size: cover ; border-radius: 5px ; height: 250px'>
    <div style="float: right ; margin: 50px ; padding: 20px ; background: rgba(255 , 255 , 255 , 0.7) ; width: 50% ; height: 150px">
        <div style="position: relative ; top: 50% ; transform: translatey(-50%)">
            <div style="font-size: xx-large ; font-weight: 900 ; color: rgba(0 , 0 , 0 , 0.8) ; line-height: 100%">Tutorial by Mondaic</div>
            <div style="font-size: large ; padding-top: 20px ; color: rgba(0 , 0 , 0 , 0.5)">For Salvus version 2025.1.2</div>
        </div>
    </div>
</div>

# Lamb's problem

## Part 4 - Heterogeneous models

In this part, we consider media with non-constant material parameters and demonstrate how external models
can be read into a project.

In [ ]:
%matplotlib inline

In [ ]:
# This notebook will use this variable to determine which
# remote site to run on.
import os

SALVUS_FLOW_SITE_NAME = os.environ.get("SITE_NAME", "local")
PROJECT_DIR = "project"

In [ ]:
import numpy as np
import xarray as xr

In [ ]:
import salvus.namespace as sn

-

## Basic project setup like in the first tutorial notebook

In [ ]:
d = sn.domain.dim2.BoxDomain(x0=0.0, x1=2000.0, y0=0.0, y1=1000.0)

p = sn.Project.from_domain(
    path="project_lambs_problem_volumetric_model",
    domain=d,
    load_if_exists=True,
)

src = sn.simple_config.source.cartesian.MomentTensorPoint2D(
    x=500.0, y=500.0, mxx=1.0, myy=1.0, mxy=0.0
)

recs = [
    sn.simple_config.receiver.cartesian.Point2D(
        y=800.0,
        x=x,
        network_code="REC",
        station_code=f"{_i:05}",
        fields=["displacement"],
    )
    for _i, x in enumerate(np.linspace(1010.0, 1410.0, 5))
]

p.add_to_project(sn.Event(event_name="event_0", sources=src, receivers=recs))

## Define an xarray model

In [ ]:
import numpy as np
import xarray as xr

# It helps to define the model larger than your domain.
nx, ny = 600, 400
x = np.linspace(-1000, 3000, nx)
y = np.linspace(-1000, 2000, ny)
xx, yy = np.meshgrid(x, y, indexing="ij")

# Background model.
vp = 3000.0 * np.ones_like(xx)
rho = 2200.0 * np.ones_like(xx)


# Sinc function.
r = ((xx - 1000.0) / 200) ** 2 + ((yy - 500) / 200) ** 2
vp -= (np.sin(r) / r) * 1000

# Use a fixed Vp/Vs ratio.
vs = vp / 1.7

ds = xr.Dataset(
    data_vars={
        "VP": (["x", "y"], vp),
        "VS": (["x", "y"], vs),
        "RHO": (["x", "y"], rho),
    },
    coords={"x": x, "y": y},
)

ds.VP.T.plot(figsize=(10, 6))

## Use it

In [ ]:
model = sn.model.volume.cartesian.GenericModel(name="volume", data=ds)
p.add_to_project(model, overwrite=True)

p.add_to_project(
    sn.SimulationConfiguration(
        name="volumetric_model",
        max_frequency_in_hertz=30.0,
        elements_per_wavelength=1.0,
        tensor_order=1,
        model_configuration=sn.ModelConfiguration(
            background_model=sn.model.background.from_volume_model(
                model=model
            ),
            volume_models=["volume"],
        ),
        event_configuration=sn.EventConfiguration(
            wavelet=sn.simple_config.stf.Ricker(center_frequency=15.0),
            waveform_simulation_configuration=sn.WaveformSimulationConfiguration(
                end_time_in_seconds=0.6
            ),
        ),
    ),
    overwrite=True,
)

p.viz.nb.simulation_setup(
    simulation_configuration="volumetric_model", events=p.events.list()
)

In [ ]:
p.simulations.launch(
    ranks_per_job=2,
    site_name=SALVUS_FLOW_SITE_NAME,
    events=p.events.list(),
    simulation_configuration="volumetric_model",
    extra_output_configuration={
        "volume_data": {
            "sampling_interval_in_time_steps": 10,
            "fields": ["displacement"],
        },
    },
)
p.simulations.query(block=True)

xdmf_path = (
    p.simulations.get_simulation_output_directory(
        simulation_configuration="volumetric_model", event=p.events.list()[0]
    )
    / "volume_data_output_elastic_volume.xdmf"
)

In [ ]:
# OSX:
# !open $xdmf_path

# Linux:
# !xdg-open $xdmf_path

## Bonus - using an image as a model

Not very scientific but some people might find this useful. This simple images has been created in Photoshop:

![](./data/salvus_image.png)

In [ ]:
# Might need to install imageio.
# !pip install imageio

In [ ]:
import imageio

image = np.require(
    imageio.imread("./data/salvus_image.png").T[:, ::-1], dtype=np.float32
)

ones = np.ones((2000, 1000), dtype=np.float32)

ds_image = xr.Dataset(
    data_vars={
        "VP": (["x", "y"], ones * 3000.0 + 2 * image),
        "VS": (["x", "y"], ones * 1847.5),
        "RHO": (["x", "y"], ones * 2200.0),
    },
    coords={"x": np.linspace(0, 2000, 2000), "y": np.linspace(0, 1000, 1000)},
)

ds_image.VP.T.plot(figsize=(10, 6))

In [ ]:
model_image = sn.model.volume.cartesian.GenericModel(
    name="volume_image", data=ds_image
)
p.add_to_project(model_image, overwrite=True)

p.add_to_project(
    sn.SimulationConfiguration(
        name="volumetric_model_image",
        max_frequency_in_hertz=30.0,
        elements_per_wavelength=2.0,
        tensor_order=4,
        model_configuration=sn.ModelConfiguration(
            background_model=sn.model.background.from_volume_model(
                model=model_image
            ),
            volume_models=["volume_image"],
        ),
        event_configuration=sn.EventConfiguration(
            wavelet=sn.simple_config.stf.Ricker(center_frequency=15.0),
            waveform_simulation_configuration=sn.WaveformSimulationConfiguration(
                end_time_in_seconds=0.6
            ),
        ),
    ),
    overwrite=True,
)

p.viz.nb.simulation_setup(
    simulation_configuration="volumetric_model_image", events=p.events.list()
)

In [ ]:
p.simulations.launch(
    ranks_per_job=2,
    site_name=SALVUS_FLOW_SITE_NAME,
    events=p.events.list(),
    simulation_configuration="volumetric_model_image",
    extra_output_configuration={
        "volume_data": {
            "sampling_interval_in_time_steps": 10,
            "fields": ["displacement"],
        },
    },
)
p.simulations.query(block=True)

xdmf_path = (
    p.simulations.get_simulation_output_directory(
        simulation_configuration="volumetric_model_image",
        event=p.events.list()[0],
    )
    / "volume_data_output_elastic_volume.xdmf"
)

In [ ]:
# OSX:
# !open $xdmf_path

# Linux:
# !xdg-open $xdmf_path